In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import pickle
import time
from tqdm import tqdm
import gc

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,SimpleImputer
from sklearn.linear_model import BayesianRidge, Ridge, HuberRegressor, RidgeCV

TUNING = False

np.random.seed(0)

***
### preparing the data

In [ ]:
root = Path("../input/janestreet-preprocessing")

train = pd.read_parquet(root/"train.parquet")
features = pd.read_parquet(root/"features.parquet")

train.info()

In [ ]:
train = train.query("date > 85").query("weight > 0").reset_index(drop=True)

In [ ]:
input_features = [col for col in train.columns if "feature" in col]
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']

X_dset = train.loc[:,input_features].copy()
y_dset = y_dset = (train.loc[:,resp_cols] > 0).astype(int).copy()

In [ ]:
X_dset.isna().sum(axis=0)

In [ ]:
# percentage of rows with at least one nan value
100 * X_dset.isna().any(axis=1).sum() / len(X_dset)

In [ ]:
# percentage of entries with nan values
100*X_dset.isna().sum().sum() / (X_dset.shape[0]*X_dset.shape[1])

In [ ]:
# feature with nan values ordered from most to least
(100*X_dset.isna().sum(axis=0)/len(X_dset)).sort_values(ascending=False).head(50)

In [ ]:
# feature without nan values
print(len(X_dset.isna().sum(axis=0)[X_dset.isna().sum(axis=0) == 0]))
X_dset.isna().sum(axis=0)[X_dset.isna().sum(axis=0) == 0]

***
### testing different imputers

In [ ]:
def l1_distance(x, y):
    return np.mean(np.abs(x-y))

def l2_distance(x, y):
    return np.sqrt(np.mean((x-y)**2))

In [ ]:
if TUNING:
    nan_perc = X_dset.isna().sum(axis=0) / len(X_dset)
    
    X = X_dset.dropna().reset_index(drop=True)
    X_nan = X.copy(deep=True)
    
    for feat,perc in nan_perc.iteritems():
        idx = X[feat].sample(frac=perc).index.values
        if len(idx)==0: continue
        X_nan.loc[idx,feat] = np.nan

    X_nan_train = X_nan.loc[:2*len(X_nan)//3,:].copy()
    X_nan_valid = X_nan.loc[2*len(X_nan)//3+1:,:].copy()
    nan_mask = np.isnan(X_nan_valid.values)

In [ ]:
if TUNING:
    # imputer for feature_0 == -1 & feature_0 == 1
    imputer_f0m1 = SimpleImputer(strategy="mean")
    imputer_f0p1 = SimpleImputer(strategy="mean")

    tic = time.time()
    idx_f0m1 = X_nan_train.query("feature_0 == -1").index
    idx_f0p1 = X_nan_train.query("feature_0 == 1").index
    imputer_f0m1.fit(X_nan_train.loc[idx_f0m1, input_features[1:]])
    imputer_f0p1.fit(X_nan_train.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Fit time: {(tac-tic)/60} min.")

    tic = time.time()
    idx_f0m1 = X_nan_valid.query("feature_0 == -1").index
    idx_f0p1 = X_nan_valid.query("feature_0 == 1").index
    X_nan_valid_ = X_nan_valid.copy(deep=True)
    X_nan_valid_.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_nan_valid_.loc[idx_f0m1, input_features[1:]])
    X_nan_valid_.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_nan_valid_.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Predict time: {(tac-tic)/60} min.")

    l1 = l1_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    l2 = l2_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    print(f"L1: {l1}  -  L2: {l2}")

    del imputer_f0m1,imputer_f0p1; gc.collect()

In [ ]:
if TUNING:
    # imputer for feature_0 == -1 & feature_0 == 1
    imputer_f0m1 = SimpleImputer(strategy="median")
    imputer_f0p1 = SimpleImputer(strategy="median")

    tic = time.time()
    idx_f0m1 = X_nan_train.query("feature_0 == -1").index
    idx_f0p1 = X_nan_train.query("feature_0 == 1").index
    imputer_f0m1.fit(X_nan_train.loc[idx_f0m1, input_features[1:]])
    imputer_f0p1.fit(X_nan_train.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Fit time: {(tac-tic)/60} min.")

    tic = time.time()
    idx_f0m1 = X_nan_valid.query("feature_0 == -1").index
    idx_f0p1 = X_nan_valid.query("feature_0 == 1").index
    X_nan_valid_ = X_nan_valid.copy(deep=True)
    X_nan_valid_.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_nan_valid_.loc[idx_f0m1, input_features[1:]])
    X_nan_valid_.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_nan_valid_.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Predict time: {(tac-tic)/60} min.")

    l1 = l1_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    l2 = l2_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    print(f"L1: {l1}  -  L2: {l2}")

    del imputer_f0m1,imputer_f0p1; gc.collect()

In [ ]:
if TUNING:
    # imputer for feature_0 == -1 & feature_0 == 1
    imputer_f0m1 = SimpleImputer(strategy="most_frequent")
    imputer_f0p1 = SimpleImputer(strategy="most_frequent")

    tic = time.time()
    idx_f0m1 = X_nan_train.query("feature_0 == -1").index
    idx_f0p1 = X_nan_train.query("feature_0 == 1").index
    imputer_f0m1.fit(X_nan_train.loc[idx_f0m1, input_features[1:]])
    imputer_f0p1.fit(X_nan_train.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Fit time: {(tac-tic)/60} min.")

    tic = time.time()
    idx_f0m1 = X_nan_valid.query("feature_0 == -1").index
    idx_f0p1 = X_nan_valid.query("feature_0 == 1").index
    X_nan_valid_ = X_nan_valid.copy(deep=True)
    X_nan_valid_.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_nan_valid_.loc[idx_f0m1, input_features[1:]])
    X_nan_valid_.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_nan_valid_.loc[idx_f0p1, input_features[1:]])
    tac = time.time()
    print(f"Predict time: {(tac-tic)/60} min.")

    l1 = l1_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    l2 = l2_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
    print(f"L1: {l1}  -  L2: {l2}")

    del imputer_f0m1,imputer_f0p1; gc.collect()

In [ ]:
# prev experiment showed that max_iter=1 gives best performance

if TUNING:

    for n_features in range(5,51,5):
        print(f" n_features: {n_features} ".center(60,"-"))

        imputer_kwargs = dict(
            estimator=BayesianRidge(normalize=True), 
            max_iter=1, 
            n_nearest_features=n_features,
            initial_strategy="median",
            imputation_order="ascending",
            skip_complete=True,
            verbose=0,
            random_state=2,
        )

        # imputer for feature_0 == -1 & feature_0 == 1
        imputer_f0m1 = IterativeImputer(**imputer_kwargs)
        imputer_f0p1 = IterativeImputer(**imputer_kwargs)

        tic = time.time()
        idx_f0m1 = X_nan_train.query("feature_0 == -1").index
        idx_f0p1 = X_nan_train.query("feature_0 == 1").index
        imputer_f0m1.fit(X_nan_train.loc[idx_f0m1, input_features[1:]])
        imputer_f0p1.fit(X_nan_train.loc[idx_f0p1, input_features[1:]])
        tac = time.time()
        print(f"Fit time: {(tac-tic)/60} min.")

        tic = time.time()
        idx_f0m1 = X_nan_valid.query("feature_0 == -1").index
        idx_f0p1 = X_nan_valid.query("feature_0 == 1").index
        X_nan_valid_ = X_nan_valid.copy(deep=True)
        X_nan_valid_.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_nan_valid_.loc[idx_f0m1, input_features[1:]])
        X_nan_valid_.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_nan_valid_.loc[idx_f0p1, input_features[1:]])
        tac = time.time()
        print(f"Predict time: {(tac-tic)/60} min.")

        l1 = l1_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
        l2 = l2_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
        print(f"L1: {l1}  -  L2: {l2}")

        del imputer_f0m1,imputer_f0p1; gc.collect()

In [ ]:
# without normalization 

imputer_kwargs = dict(
    estimator=BayesianRidge(normalize=False), 
    max_iter=1, 
    n_nearest_features=30,
    initial_strategy="median",
    imputation_order="ascending",
    skip_complete=True,
    verbose=0,
    random_state=2,
)

# imputer for feature_0 == -1 & feature_0 == 1
imputer_f0m1 = IterativeImputer(**imputer_kwargs)
imputer_f0p1 = IterativeImputer(**imputer_kwargs)

tic = time.time()
idx_f0m1 = X_nan_train.query("feature_0 == -1").index
idx_f0p1 = X_nan_train.query("feature_0 == 1").index
imputer_f0m1.fit(X_nan_train.loc[idx_f0m1, input_features[1:]])
imputer_f0p1.fit(X_nan_train.loc[idx_f0p1, input_features[1:]])
tac = time.time()
print(f"Fit time: {(tac-tic)/60} min.")

tic = time.time()
idx_f0m1 = X_nan_valid.query("feature_0 == -1").index
idx_f0p1 = X_nan_valid.query("feature_0 == 1").index
X_nan_valid_ = X_nan_valid.copy(deep=True)
X_nan_valid_.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_nan_valid_.loc[idx_f0m1, input_features[1:]])
X_nan_valid_.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_nan_valid_.loc[idx_f0p1, input_features[1:]])
tac = time.time()
print(f"Predict time: {(tac-tic)/60} min.")

l1 = l1_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
l2 = l2_distance(X_nan_valid_.values[nan_mask], X.loc[X_nan_valid.index].values[nan_mask])
print(f"L1: {l1}  -  L2: {l2}")

del imputer_f0m1,imputer_f0p1; gc.collect()

***
### fitting the imputer

In [ ]:
imputer_kwargs = dict(
    estimator=BayesianRidge(normalize=True), 
    max_iter=1, 
    n_nearest_features=30,
    initial_strategy="median",
    imputation_order="ascending",
    skip_complete=True,
    verbose=1,
    random_state=2,
)

# imputer for feature_0 == -1 & feature_0 == 1
imputer_f0m1 = IterativeImputer(**imputer_kwargs)
imputer_f0p1 = IterativeImputer(**imputer_kwargs)

In [ ]:
%%time
imputer_f0m1.fit(X_dset.query("feature_0 == -1").loc[:, input_features[1:]])

In [ ]:
%%time
imputer_f0p1.fit(X_dset.query("feature_0 == 1").loc[:, input_features[1:]])

In [ ]:
# testing inference time
X_sample = X_dset[X_dset.isna().any(axis=1)].sample(5000)

In [ ]:
all_times = list()
imputer_f0m1.verbose = False
imputer_f0p1.verbose = False

for i,row in tqdm(X_sample.iterrows()):
    tic = time.time()
    if row.feature_0 < 0:
        _ = imputer_f0m1.transform(row.values[1:].reshape(1,-1))
    else:
        _ = imputer_f0p1.transform(row.values[1:].reshape(1,-1))
    tac = time.time()
    all_times.append(tac-tic)

print("Inference-time per sample:", np.mean(np.asarray(all_times)*1000), "[ms]")

In [ ]:
len(imputer_f0m1.imputation_sequence_)

In [ ]:
len(imputer_f0p1.imputation_sequence_)

In [ ]:
with open("imputer_f0m1.pickle", "wb") as file:
    pickle.dump(imputer_f0m1, file, protocol=pickle.HIGHEST_PROTOCOL)
    file.close()
    
with open("imputer_f0p1.pickle", "wb") as file:
    pickle.dump(imputer_f0p1, file, protocol=pickle.HIGHEST_PROTOCOL)
    file.close()

***